# Notebook de benchmarking de la Ecuación de Poisson
[Documentación](#https://juliaci.github.io/BenchmarkTools.jl/stable/manual/)

In [1]:
using BenchmarkTools, Statistics, Plots

In [2]:
plotly()

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/oscar/.julia/packages/Plots/9C6z9/src/backends.jl:432


Plots.PlotlyBackend()

In [3]:
include("FVM.jl")

Main.FVM

## Ecuación de Poisson 3D

### Construcción de la malla

In [4]:
function benchmark_mesh(vols::Int)
    volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces = FVM.uniform_grid(vols, vols/10)
    tags = FVM.init_tags(3, volumes, centers_and_boundaries)
    tags_b = FVM.init_tags_boundaries(3, centers_and_boundaries)

    FVM.tag_wall(tags, tags_b, [:W, :E, :T, :N, :B], 0, :D)
    FVM.tag_wall(tags, tags_b, :S, 100, :D)

    mesh = FVM.Mesh(volumes, lengths, centers, centers_and_boundaries, deltas, faces, deltas_faces, tags, tags_b);
end

benchmark_mesh (generic function with 1 method)

In [ ]:
vols = [10, 20, 50, 100]
times_mesh = []
for vol ∈ vols
    println(vol)
    t = @benchmark benchmark_mesh($vol)
    push!(times_mesh, t)
end

In [5]:
times_mesh[1]

BenchmarkTools.Trial: 347 samples with 1 evaluation.
 Range (min … max):  13.352 ms … 19.920 ms  ┊ GC (min … max): 0.00% … 24.81%
 Time  (median):     13.563 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.417 ms ±  1.768 ms  ┊ GC (mean ± σ):  4.91% ±  8.74%

  ▃█▇▃▂                                             ▁          
  ███████▇█▁▄▁▁▄▅▁▄▄▅▁▄▁▄▄▁▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇████▄▇▆▆▆ ▆
  13.4 ms      Histogram: log(frequency) by time      18.8 ms <

 Memory estimate: 7.27 MiB, allocs estimate: 77869.

In [6]:
means_mesh = [mean(time).time*10^-6 for time ∈ times_mesh]

4-element Vector{Float64}:
    14.416873948126806
   115.65228797727278
  2187.136232333334
 17220.691974000005

In [7]:
stds_mesh = [std(time).time*10^-6 for time ∈ times_mesh]

4-element Vector{Float64}:
   1.7684560442438453
   6.51673457844921
 273.8571146355588
 NaN

In [11]:
plot(vols, means_mesh, ribbons=stds_mesh, fillalpha=.5)

In [80]:
#times_mesh = Dict(10 => "13.66 ms", 20 => "109.934 ms", 30 => "396.965 ms", 50 => "2.000 s")

### Haciendo cuentitas del FVM

In [5]:
function Γ_constant(x::Array, y::Array, z::Array)
    Γ = 1000
    tensor_Γ = ones(length(x), length(y), length(z))
    tensor_Γ = Γ .* tensor_Γ
end

Γ_constant (generic function with 1 method)

In [6]:
function benchmark_set_boundary_conditions(mesh::FVM.Mesh)
    coeff = FVM.init_coefficients(mesh)
    FVM.set_diffusion(coeff, Γ_constant);
    return coeff
end

benchmark_set_boundary_conditions (generic function with 1 method)

In [14]:
vols = [10, 20, 50, 100]
times_boundary_conditions = []
for vol ∈ vols
    println(vol)
    mesh = benchmark_mesh(vol)
    t = @benchmark benchmark_set_boundary_conditions($mesh)
    push!(times_boundary_conditions, t)
end

10
20
50
100


In [15]:
times_boundary_conditions[1]

BenchmarkTools.Trial: 811 samples with 1 evaluation.
 Range (min … max):  5.313 ms … 14.879 ms  ┊ GC (min … max):  0.00% … 45.36%
 Time  (median):     5.419 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.153 ms ±  2.141 ms  ┊ GC (mean ± σ):  10.75% ± 15.84%

  █▆                                                      ▁▃  
  ███▆▇▄▁▄▁▄▄▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▄██ ▇
  5.31 ms      Histogram: log(frequency) by time     12.6 ms <

 Memory estimate: 4.40 MiB, allocs estimate: 89480.

In [16]:
means_boundary_conditions = [mean(time).time*10^-6 for time ∈ times_boundary_conditions]

4-element Vector{Float64}:
    6.152931173859435
   36.66016913868614
  302.28351964705894
 1650.8132627500006

In [17]:
stds_boundary_conditions = [std(time).time*10^-6 for time ∈ times_boundary_conditions]

4-element Vector{Float64}:
  2.140563114387661
  3.173854532820231
 15.061169946796836
 39.262702735527036

In [18]:
plot(vols, means_boundary_conditions, ribbons=stds_boundary_conditions, fillalpha=.5)

In [84]:
#times_setting_boundary_conditions = Dict(10 => "5.593 ms", 20 => "35.935 ms", 30 => "96.358 ms", 50 => "300.607 ms")

### Obteniendo soluciones

In [7]:
function benchmark_solutions(coeff::FVM.Coefficients_3d)
    equation_system = FVM.init_eq_system(coeff)
    solution = FVM.get_solution(equation_system);
end

benchmark_solutions (generic function with 1 method)

In [8]:
vols = [10, 20]
times_solution = []
for vol ∈ vols
    println(vol)
    mesh = benchmark_mesh(vol)
    coeff = benchmark_set_boundary_conditions(mesh)
    t = @benchmark benchmark_solutions($coeff)
    push!(times_solution, t)
end

10
20


In [13]:
mesh = benchmark_mesh(50)
coeff = benchmark_set_boundary_conditions(mesh)
t = @benchmark benchmark_solutions(coeff)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 15.328 s (0.06% GC) to evaluate,
 with a memory estimate of 3.31 GiB, over 3000503 allocations.

In [ ]:
mesh = benchmark_mesh(100)
coeff = benchmark_set_boundary_conditions(mesh)
t = @benchmark benchmark_solutions(coeff)

In [9]:
times_solution[1]

BenchmarkTools.Trial: 682 samples with 1 evaluation.
 Range (min … max):  6.344 ms …  15.109 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.046 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.303 ms ± 814.024 μs  ┊ GC (mean ± σ):  2.88% ± 6.93%

         ▁▁▁ ▆█▅▁                                         ▂▁   
  ▄▁▁▁▁▁▁███▇█████▇▆▁▅▅▅▁▁▄▁▁▁▁▁▁▁▁▄▄▁▁▅▄▁▁▄▁▁▁▁▁▁▁▁▁▄▆▆▄▄███ ▇
  6.34 ms      Histogram: log(frequency) by time       9.7 ms <

 Memory estimate: 4.33 MiB, allocs estimate: 24325.

In [10]:
means_solution = [mean(time).time*10^-6 for time ∈ times_solution]

2-element Vector{Float64}:
   7.302924491202349
 122.2876960243903

In [11]:
stds_solution = [std(time).time*10^-6 for time ∈ times_solution]

2-element Vector{Float64}:
 0.8140243320932815
 2.112067293004304

In [12]:
plot(vols, means_solution, ribbons=stds_solution, fillalpha=.5)

In [83]:
#times_solutions = Dict(10 => "6.398 ms", 20 => "116.080 ms", 30 => "934.002 ms", 50 => "15.803 s")

### Graficando

In [18]:
#FVM.plot_solution(solution, mesh)

---